<a href="https://colab.research.google.com/github/bansalsparsh/TaleSpeak/blob/main/TaleSpeak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
# --------------------------
# 0. Install Dependencies
# --------------------------
!pip install edge-tts nltk pydub nest_asyncio

In [80]:
# --------------------------
# 1. Environment Setup
# --------------------------
import nltk, os, nest_asyncio, edge_tts
from nltk.tokenize import sent_tokenize
from pydub import AudioSegment

nltk.download('punkt')
nest_asyncio.apply()
os.makedirs("audio_parts", exist_ok=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [81]:
# --------------------------
# 2. Adjusting Rate of Speech
# --------------------------
emotion_keywords = {
    'cheerful': ['joy','smile','wonderful','excited','love'],
    'sad': ['cry','sad','tears','gloomy','sorrow'],
    'angry': ['angry','mad','furious','shout','rage'],
    'excited': ['wow','suddenly','amazing','surprise','shock'],
}

# Emotion → speed adjustments
emotion_rate = {
    'cheerful': '+15%',
    'sad': '-20%',
    'angry': '+25%',
    'excited': '+20%',
    None: '+0%'
}

In [82]:
# --------------------------
# 3. Text Processing
# --------------------------
story = """
It was a bright morning in the village.
Lina, a little girl, was playing outside her house.
Suddenly, she heard a soft whimper behind a bush.
She walked slowly and peeked inside.
A tiny puppy sat there, shivering.
Its fur was muddy, and it looked scared.
“Oh no, are you lost?” Lina asked gently.
The puppy wagged its tail and barked softly.
Without thinking twice, Lina picked it up and hugged it tightly.
“I’ll take care of you,” she whispered.
That evening, Lina put up posters around the village.
Soon, an old man came running.
“That’s my Tommy!” he cried with joy.
Lina smiled, even though she felt a little sad.
She handed the puppy back.
Tommy licked her cheek before leaving, as if saying thank you.
From that day on, Lina and the old man became good friends.
And sometimes, Tommy still visited — tail wagging, happy and free.
"""

sentences = sent_tokenize(story)

In [83]:
# --------------------------
# 4. Emotion Detection System
# --------------------------
def detect_emotion(s):
    scores = {e: 0 for e in emotion_keywords}
    for w in s.lower().split():
        w = w.strip(".,!?")
        for e, kws in emotion_keywords.items():
            if w in kws:
                scores[e] += 1
    top = max(scores, key=scores.get)
    return top if scores[top] > 0 else None

In [84]:
# --------------------------
# 5. Enhanced Story Narrator
# --------------------------
async def generate():
    for i, s in enumerate(sentences):
        emo = detect_emotion(s)
        rate = emotion_rate[emo]
        print(f"[{emo or 'neutral'}] {s}")

        communicator = edge_tts.Communicate(
            text=s,
            voice="en-US-JennyNeural",
            rate=rate
        )
        out = f"audio_parts/part_{i}.mp3"
        await communicator.save(out)
    print("✅ Generated audio parts.")

# Run
await generate()

# Combine into a single WAV file
final = AudioSegment.empty()
for i in range(len(sentences)):
    final += AudioSegment.from_file(f"audio_parts/part_{i}.mp3", format="mp3")
final.export("output.wav", format="wav")
print("output.wav is ready.")


from IPython.display import Audio
Audio("output.wav", autoplay=True)

[neutral] 
It was a bright morning in the village.
[neutral] Lina, a little girl, was playing outside her house.
[excited] Suddenly, she heard a soft whimper behind a bush.
[neutral] She walked slowly and peeked inside.
[neutral] A tiny puppy sat there, shivering.
[neutral] Its fur was muddy, and it looked scared.
[neutral] “Oh no, are you lost?” Lina asked gently.
[neutral] The puppy wagged its tail and barked softly.
[neutral] Without thinking twice, Lina picked it up and hugged it tightly.
[neutral] “I’ll take care of you,” she whispered.
[neutral] That evening, Lina put up posters around the village.
[neutral] Soon, an old man came running.
[cheerful] “That’s my Tommy!” he cried with joy.
[sad] Lina smiled, even though she felt a little sad.
[neutral] She handed the puppy back.
[neutral] Tommy licked her cheek before leaving, as if saying thank you.
[neutral] From that day on, Lina and the old man became good friends.
[neutral] And sometimes, Tommy still visited — tail wagging, hap